### All import statements

In [1]:
import pandas as pd
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
import cv2
import numpy as np

Using TensorFlow backend.
/home/chocolatethunder/anaconda3/envs/ML/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Variables for training

outputsz = 2 
n_epoch = 2
img_size = 299
batch_size = 32
train_dir = "train/"
val_dir = "val/"
n_images = 25000

### Preprocessing Images

#### Converting images to numpy arrays (takes a lot of memory so not recommended)

In [ ]:
image_path_list = []
valid_image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"]
valid_image_extensions = [item.lower() for item in valid_image_extensions]
    
for file in os.listdir(imageDir):
    extension = os.path.splitext(file)[1]
    if extension.lower() not in valid_image_extensions:
        continue
    image_path_list.append(os.path.join(imageDir, file))

img_num = len(image_path_list)
    
x = np.zeros((10000,img_sz,img_sz,3))
y = np.zeros((img_num))

for ii in range(10000):
    im = cv2.imread(image_path_list[ii])
    resized_image = cv2.resize(im, (img_sz, img_sz))
    x[ii,:,:,:] = resized_image
    
    if image_path_list[0].find("dog") >=0:
        y[ii] = 1
    else:
        y[ii] =0

#### Creating generator to get images from directory 

In [7]:
#preprocess_input scales and centers the images sample wise
# this function also adds random noise to the data by rotations, shifts, zooms and shears
train_datagen =  image.ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)


#create generators for data
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(img_size, img_size),
batch_size=batch_size,
)

val_datagen = image.ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)


#val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
  )

Found 25000 images belonging to 2 classes.


### Create model with a base  from inceptionv3 weights from imagenet

In [8]:
#include_top = False means that we do not take the final layers from InceptionV3
Iv3 = InceptionV3(weights='imagenet', include_top=False)

# add a final layers to the model, do Global Average Pooling, Dense and FC
x =Iv3.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
y = Dense(outputsz, activation='softmax')(x)
#create model
model = Model(input=Iv3.input, output=y)

/home/chocolatethunder/anaconda3/envs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


### Transfer Learning

In [9]:
#at first we only train the added layers so we freeze all the layers below
for layer in Iv3.layers:
    layer.trainable = False

#use categorical loss entropy because it is equivalent to log loss which is the metric of the competition
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy')

#train the model
transfer_learn = model.fit_generator(generator = train_generator, 
                                     steps_per_epoch = n_images/batch_size,
                                     epochs = n_epoch,
                                     validation_data = val_generator,
                                     validation_steps = 156)

Epoch 1/2
625/625 [==============================] - 18242s 29s/step - loss: 0.1772
Epoch 2/2
625/625 [==============================] - 10693s 17s/step - loss: 0.1161


### Fine Tune

#### Once the last layers have been trained we go back and fine tune some of the levels of the model. We use a low training rate since we assume that the features are good and we do not want them to change very much

In [11]:
# unfreeze the last two blocks so we can train them
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
   
#finetune with a smaller learning rate, we want to make sure things change quite slowly 
model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy')

#train the model
transfer_learn = model.fit_generator(generator = train_generator, 
                                     steps_per_epoch = n_images/batch_size,
                                     epochs = n_epoch,
                                     validation_data = val_generator,
                                     validation_steps = 156)


Epoch 1/2
625/625 [==============================] - 67309s 108s/step - loss: 0.0752
Epoch 2/2
625/625 [==============================] - 69730s 112s/step - loss: 0.0471


### Train on the remaning validation images

In [ ]:
#once we see it does not overfit than we train on the remaining validation images for one 
#epoch
transfer_learn = model.fit_generator(generator = val_generator, 
                                     steps_per_epoch = 5000/batch_size,
                                     epochs = 1)

### Save the model

In [12]:
model.save('transfer_learning1.h5')


### Model Predictions

In [14]:


#predict all of the test images
#for some reason using the fit_generator even with shuffle = false, shuffled the images so instead
#I had to take them out image by image to be predicted, this is inefficient but it didn't work if I did
# it by a batch at a time either
predict = np.zeros((12500,2))
for ii in range(12500):
    img_dir = "test/T/{0}.jpg".format(ii+1)
    im = cv2.imread(img_dir)
    resized_image = cv2.resize(im, (img_size, img_size))
    resized_image = resized_image/127.5
    resized_image -= 1.
    batch_img = np.zeros((1,299,299,3))
    batch_img[0,:,:,:] = resized_image
    predict[ii,:] = model.predict(batch_img)



In [25]:
#take only predictions for dogs
dog = predict[:,1]
#clip values
dog_clip = np.clip(dog, a_min = 0.005, a_max = 0.99)

### Save predictions to CSV

#### function to create csv files from the predictions

In [15]:

def write_preds(preds, fname):
    pd.DataFrame({"Id": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)


In [26]:
#write to csv
write_preds(dog, "dog.csv")
write_preds(dog_clip, "dog_clip.csv")